In [1]:
# import des bibliothèques nécessaires
import cv2

# Chemin vers le fichier vidéo  
video_path = r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon\demi_finale_S2_250_Dijon_novembre_2025.mp4'

# Vitesse de lecture de la vidéo (1=normale, 0=pause, >1=plus rapide)
play_speed = 2


Encodage en direct des 'Points' et 'Temps mort' à partir de la lecture de la vidéo

In [2]:
# Liste pour stocker les actions
actions = []

# Dictionnaire des touches -> action
key_action_map = {
    ord('1'): 'Service J&M',
    ord('2'): 'Temps hors-jeu',
    # ord('3'): 'Service adversaire',
    # ord('7') : 'Début de set',
}

# Ouvrir la vidéo
cap = cv2.VideoCapture(video_path)

def _waitKey_fast(ms):
    # réduire le délai proportionnellement à la vitesse (au moins 1 ms)
    adj = max(1, int(ms / play_speed))
    return cv2.waitKey(adj)

if not cap.isOpened():
    print("Erreur : impossible d’ouvrir la vidéo.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
frame_number = 0

print("\n--- LECTURE EN COURS ---\nAppuie sur une touche pour taguer une action : 1/2 (ou q pour quitter)\n")
paused = False
last_action = ""  # Variable pour stocker la dernière action

try:
    while cap.isOpened():
        if not paused:
            ret, frame = cap.read()
            if not ret:
                print("Fin de la vidéo ou erreur de lecture.")
                break
            frame_number += 1

        # Affiche la dernière action
        if last_action and ret:
            cv2.putText(frame,
                        f"Derniere action : {last_action}",
                        (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 255, 0),
                        2,
                        cv2.LINE_AA)

        # Indiquer mode pause
        if paused and ret:
            cv2.putText(frame,
                        "|| PAUSE ||",
                        (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 0, 255),
                        2,
                        cv2.LINE_AA)

        if ret:
            cv2.imshow('Beach Volley Video', frame)


        key = _waitKey_fast(30) & 0xFF
        if key == ord('q'):  # touche 'q' pour quitter
            break
        elif key == ord(' '):  # barre espace
            paused = not paused
        elif key in key_action_map:
            timestamp = frame_number / fps
            timestamp_hhmmss = f"{int(timestamp // 3600):02d}:{int((timestamp % 3600) // 60):02d}:{int(timestamp % 60):02d}"
            action_name = key_action_map[key]
            last_action = action_name

            actions.append({
                # 'Time (MM:SS)': timestamp_hhmmss,
                'Frame': frame_number,
                # 'Time (s)': round(timestamp, 2),
                'Action': action_name
            })

finally:
    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()


### Ecriture des intervalles en numéro de frames dans deux fichiers textes : 1 pour les points, 1 pour les temps hors-jeu

## Fichier 'Points.txt' (format : frame_start-frame_end)
points_file = 'points_frames.txt'
# Construire les intervalles : 1 ligne = time(Point) - time(Temps hors-jeu) suivant
points_frames = []
i = 0
while i < len(actions):
    if actions[i]['Action'] == 'Point':
        start = actions[i]['Frame']
        # chercher le prochain 'Temps hors-jeu'
        j = i + 1
        end = None
        while j < len(actions):
            if actions[j]['Action'] == 'Temps hors-jeu':
                end = actions[j]['Frame']
                break
            j += 1
        if end is not None:
            points_frames.append((start, end))
            i = j + 1
        else:
            print(f"⚠️ Pas d'action 'Point' trouvée après {start:.2f}s — intervalle ignoré.")
            break
    else:
        i += 1
# Écrire les intervalles dans points_frames.txt (format : start-end))
with open(points_file, 'w', encoding='utf-8') as f:

    for start, end in points_frames:
        f.write(f"{start}-{end}\n")
print(f"✅ Fichier texte sauvegardé : {points_file}")

## Fichier 'Temps_hors_jeu.txt' (format : frame_start-frame_end)
temps_hors_jeu_file = 'temps_hors_jeu_frames.txt'
# Construire les intervalles : 1 ligne = time(Temps hors-jeu) - time(Point) suivant
temps_hors_jeu_frames = []
i = 0
while i < len(actions):
    if actions[i]['Action'] == 'Temps hors-jeu':
        start = actions[i]['Frame']
        # chercher le prochain 'Point'
        j = i + 1
        end = None
        while j < len(actions):
            if actions[j]['Action'] == 'Point':
                end = actions[j]['Frame']
                break
            j += 1
        if end is not None:
            temps_hors_jeu_frames.append((start, end))
            i = j + 1
        else:
            print(f"⚠️ Pas d'action 'Point' trouvée après {start:.2f}s — intervalle ignoré.")
            break
    else:
        i += 1
# Écrire les intervalles dans temps_hors_jeu_frames.txt (format : start-end)
with open(temps_hors_jeu_file, 'w', encoding='utf-8') as f:
    for start, end in temps_hors_jeu_frames:
        f.write(f"{start}-{end}\n")
print(f"✅ Fichier texte sauvegardé : {temps_hors_jeu_file}")



--- LECTURE EN COURS ---
Appuie sur une touche pour taguer une action : 1/2 (ou q pour quitter)

✅ Fichier texte sauvegardé : points_frames.txt
✅ Fichier texte sauvegardé : temps_hors_jeu_frames.txt


Test avec encodage "Service J&M" vs "Service adv"


In [ ]:
actions
points_frames

[(549, 899), (997, 1308), (1571, 1728)]

In [ ]:
# Liste pour stocker les actions
actions = []

# Dictionnaire des touches -> action
key_action_map = {
    ord('1'): 'Service J&M',
    ord('2'): 'Temps hors-jeu',
    ord('3'): 'Service adversaire',
    ord('7') : 'Début de set',
}

# Ouvrir la vidéo
cap = cv2.VideoCapture(video_path)

def _waitKey_fast(ms):
    # réduire le délai proportionnellement à la vitesse (au moins 1 ms)
    adj = max(1, int(ms / play_speed))
    return cv2.waitKey(adj)

if not cap.isOpened():
    print("Erreur : impossible d’ouvrir la vidéo.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
frame_number = 0

print("\n--- LECTURE EN COURS ---\nAppuie sur une touche pour taguer une action : 1/2 (ou q pour quitter)\n")
paused = False
last_action = ""  # Variable pour stocker la dernière action

try:
    while cap.isOpened():
        if not paused:
            ret, frame = cap.read()
            if not ret:
                print("Fin de la vidéo ou erreur de lecture.")
                break
            frame_number += 1

        # Affiche la dernière action
        if last_action and ret:
            cv2.putText(frame,
                        f"Derniere action : {last_action}",
                        (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 255, 0),
                        2,
                        cv2.LINE_AA)

        # Indiquer mode pause
        if paused and ret:
            cv2.putText(frame,
                        "|| PAUSE ||",
                        (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 0, 255),
                        2,
                        cv2.LINE_AA)

        if ret:
            cv2.imshow('Beach Volley Video', frame)


        key = _waitKey_fast(30) & 0xFF
        if key == ord('q'):  # touche 'q' pour quitter
            break
        elif key == ord(' '):  # barre espace
            paused = not paused
        elif key in key_action_map:
            timestamp = frame_number / fps
            timestamp_hhmmss = f"{int(timestamp // 3600):02d}:{int((timestamp % 3600) // 60):02d}:{int(timestamp % 60):02d}"
            action_name = key_action_map[key]
            last_action = action_name

            actions.append({
                # 'Time (MM:SS)': timestamp_hhmmss,
                'Frame': frame_number,
                # 'Time (s)': round(timestamp, 2),
                'Action': action_name
            })

finally:
    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()


### Création d'une liste 'points_frames' contenant les intervalles de frames correspondant aux points, avec l'information du point gagné (service précédent)


# Construire les intervalles : 1 ligne = time(Point) - time(Temps hors-jeu) suivant
points_frames = []
i = 0
while i < len(actions):
    if (actions[i]['Action'] == 'Service J&M') or (actions[i]['Action'] == 'Service adversaire'):
        start = actions[i]['Frame']
        # chercher le prochain 'Temps hors-jeu'
        j = i + 1
        end = None
        while j < len(actions):
            if actions[j]['Action'] == 'Temps hors-jeu':
                end = actions[j]['Frame']
                break
            j += 1
        if end is not None:
            points_frames.append((start, end))
            i = j + 1
        else:
            print(f"⚠️ Pas d'action 'Service' trouvée après {start:.2f}s — intervalle ignoré.")
            break
    else:
        i += 1




--- LECTURE EN COURS ---
Appuie sur une touche pour taguer une action : 1/2 (ou q pour quitter)



Découpage des segments à partir d'un fichier .txt contenant les paires de frames "Point-Temps mort"

In [ ]:
# actions.insert(1, {'Frame': 200, 'Action': 'Temps hors-jeu'})
actions

[{'Frame': 1334, 'Action': 'Service adversaire'},
 {'Frame': 1529, 'Action': 'Temps hors-jeu'},
 {'Frame': 1793, 'Action': 'Service J&M'},
 {'Frame': 2015, 'Action': 'Temps hors-jeu'}]

In [ ]:
import pandas as pd

# Créer le dataframe à partir de 'actions'
list_actions = list()

i = 0
while i < len(actions):
    action = actions[i]
    
    # Si l'action n'est pas 'Temps hors-jeu', on la traite
    if action['Action'] != 'Temps hors-jeu':
        start_frame = action['Frame']
        service_side = action['Action']
        
        # Chercher le prochain 'Temps hors-jeu'
        end_frame = None
        j = i + 1
        while j < len(actions):
            if actions[j]['Action'] == 'Temps hors-jeu':
                end_frame = actions[j]['Frame']
                break
            j += 1
        
        # Ajouter à la liste si on a trouvé un 'Temps hors-jeu'
        if end_frame is not None:
            list_actions.append({
                'Service_side': service_side,
                'Start_frame': start_frame,
                'End_frame': end_frame
            })
    
    i += 1

# Convertir en DataFrame pandas
df_actions = pd.DataFrame(list_actions)

# Ajouter les colonnes de score
df_actions['Score_J&M'] = 0
df_actions['Score_adversaire'] = 0

# Mettre à jour les scores en fonction du service_side
for idx, row in df_actions.iterrows():
    if idx == 0:
        # Première ligne : initialiser selon le service
        if row['Service_side'] == 'Service J&M':
            df_actions.at[idx, 'Score_J&M'] = 1
            df_actions.at[idx, 'Score_adversaire'] = 0
        elif row['Service_side'] == 'Service adversaire':
            df_actions.at[idx, 'Score_J&M'] = 0
            df_actions.at[idx, 'Score_adversaire'] = 1
    else:
        # Lignes suivantes : reprendre le score précédent et incrémenter selon le service
        df_actions.at[idx, 'Score_J&M'] = df_actions.at[idx - 1, 'Score_J&M']
        df_actions.at[idx, 'Score_adversaire'] = df_actions.at[idx - 1, 'Score_adversaire']
        
        if row['Service_side'] == 'Service J&M':
            df_actions.at[idx, 'Score_J&M'] += 1
        elif row['Service_side'] == 'Service adversaire':
            df_actions.at[idx, 'Score_adversaire'] += 1
df_actions


# Sauvegarder le DataFrame en fichier CSV
df_actions.to_csv('actions.csv', index=False, encoding='utf-8')
print("✅ Fichier CSV sauvegardé : actions.csv")

✅ Fichier CSV sauvegardé : actions.csv


In [ ]:
import pandas as pd
df_actions=pd.read_csv('actions.csv')
df_actions

,Service_side,Start_frame,End_frame,Score_J&M,Score_adversaire
0,Service adversaire,1334,1529,0,1
1,Service J&M,1793,2015,1,1


In [ ]:
from montage_auto_frames_gpu import extract_segments_from_df_gpu


In [ ]:
video_path

'C:\\Users\\habib\\Desktop\\Montages volley et beach\\Jade&Math\\matchs bruts\\2025 nov - S2 250 Dijon\\demi_finale_S2_250_Dijon_novembre_2025.mp4'

In [4]:
import sys
import os
import cv2
import pandas as pd


Index du dernier frame : 42861


In [6]:
def cv2_actions_to_operate(
        video_path : str,
        play_speed : float = 1.0,
    ) -> dict:
    
    """
    Enregistre les actions de montage à effectuer pour un pre process de video
    
    Args:
        video_path (str): Chemin vers la vidéo à traiter.
        play_speed (float, optional): Vitesse de lecture de la vidéo (1=normale, 0=pause, >1=plus rapide). Par défaut à 1.0.
    Returns:
        montage_actions(dict) : Dictionnaire d'actions taguées à effectuer sur la vidéo avec les éléments 'Start_frame', 'Last_frame', 'Rotation_state'
    """
    montage_actions = dict()

    # Afficher les touches disponibles en overlay sur la vidéo
    help_lines = [
        "Touches :",
        "q : quitter",
        "espace : pause/reprise",
        "0 : debut du match",
        "+ : vitesse +",
        "- : vitesse -",
        "r : rotation droite",
        "l : rotation gauche",
    ]

    _orig_imshow = cv2.imshow

    def _imshow_with_help(winname, frame):
        if frame is not None:
            x, y = 30, 120
            for i, line in enumerate(help_lines):
                cv2.putText(frame,
                            line,
                            (x, y + i * 25),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.7,
                            (255, 255, 255),
                            2,
                            cv2.LINE_AA)
        _orig_imshow(winname, frame)

    cv2.imshow = _imshow_with_help


    # Ouvrir la vidéo
    cap = cv2.VideoCapture(video_path)

    # Récupérer le nombre total de frames pour calculer le dernier frame
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    last_game_frame = frame_count - 1 if frame_count > 0 else None
    print(f"Index du dernier frame : {last_game_frame}")

    def _waitKey_fast(ms):
        # Réduire le délai proportionnellement à la vitesse (au moins 1 ms)
        adj = max(1, int(ms / play_speed))
        return cv2.waitKey(adj)

    # Vérifier que la vidéo est bien ouverte
    if not cap.isOpened():
        print("Erreur : impossible d’ouvrir la vidéo.")
        sys.exit()

    # Récupérer les FPS pour convertir les frames en temps
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_number = 0

    # État de pause et rotation
    paused = False
    rotation_state = 0  # 0, 90, 180, 270

    try:
        while cap.isOpened():

            # Lire une frame seulement si on n'est pas en pause
            if not paused:
                ret, frame = cap.read()

                # Appliquer la rotation si nécessaire
                if rotation_state == 90:
                    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
                elif rotation_state == 180:
                    frame = cv2.rotate(frame, cv2.ROTATE_180)
                elif rotation_state == 270:
                    frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

                # Afficher la vitesse de lecture en overlay
                cv2.putText(frame,
                            f"Vitesse de lecture : x{play_speed:.1f}",
                            (30, 50),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 255, 0),
                            2,
                            cv2.LINE_AA)

                # Arrêter si fin de vidéo ou erreur
                if not ret:
                    print("Fin de la vidéo ou erreur de lecture.")
                    break

                # Incrémenter le compteur de frames
                frame_number += 1

            # Indiquer le mode pause sur l'image affichée
            if paused and ret:
                cv2.putText(frame,
                            "|| PAUSE ||",
                            (30, 80),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 0, 255),
                            2,
                            cv2.LINE_AA)

            # Afficher la frame courante
            if ret:
                cv2.imshow(f'{video_path}', frame)

            # Gestion des entrées clavier
            key = _waitKey_fast(30) & 0xFF
            if key == ord('q'):  # quitter
                break
            elif key == ord(' '):  # pause/reprise
                paused = not paused
            elif key == ord('0'):  # marquer le début du match
                starting_game_frame = frame_number
                start_time = starting_game_frame / fps
                print(f"Début du match marqué au frame {starting_game_frame}, soit {start_time:.2f} secondes")
                break
            elif key == ord('+'):  # augmenter la vitesse
                play_speed += 0.5
                continue
            elif key == ord('-'):  # diminuer la vitesse
                play_speed = max(0.5, play_speed - 0.5)
                continue
            elif key == ord('r'):  # rotation à droite
                rotation_state = (rotation_state + 90) % 360
                continue
            elif key == ord('l'):  # rotation à gauche
                rotation_state = (rotation_state - 90) % 360
                continue


    finally:
        # Libérer les ressources OpenCV
        cap.release()
        cv2.destroyAllWindows()

    montage_actions = {
        'Start_frame': starting_game_frame,
        'Last_frame': last_game_frame,
        'Rotation_state': rotation_state
    }

    return montage_actions


test_montage_actions = cv2_actions_to_operate(
    video_path=r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon\demi_finale_S2_250_Dijon_novembre_2025.mp4',
    play_speed=2.0
)

test_montage_actions

Index du dernier frame : 42861
Début du match marqué au frame 148, soit 5.20 secondes


{'Start_frame': 148, 'Last_frame': 42861, 'Rotation_state': 180}

In [ ]:
def pre_match_editing(video_dir: str,
                      play_speed: float = 1.0,
                      output_dir: str = None) -> None:
    """ Réalise le découpage pré-match des vidéos d'un dossier.

    Le script utilise OpenCV pour afficher la vidéo et détecter la touche pressée.
    L'utilisateur appuie sur '0' pour indiquer le début du match, et la vidéo est ensuite découpée à partir de ce point en utilisant ffmpeg.

    Args:
        video_dir (str): dossier contenant la/les vidéo(s) à découper.
        play_speed (float): Vitesse de lecture de la vidéo.
        output_dir (str, optional): Dossier de sortie pour les vidéos découpées. Si None, les vidéos découpées seront enregistrées dans le même dossier que les vidéos d'origine.
    """

    # Initialiser un df pandas pour stocker les informations de découpage de chaque vidéo (start frame, last frame, rotation)
    match_info_df = pd.DataFrame(columns=["video_path", "starting_game_frame", "last_game_frame", "output_dir", "rotation_state"])

    # Lister les fichiers vidéo dans le dossier
    video_files = [f for f in os.listdir(video_dir) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    if not video_files:
        print("Aucune vidéo trouvée dans le dossier spécifié.")
        return
    
    # Appliquer cv2_actions_to_operate() à chaque vidéo pour récupérer les actions de montage à effectuer (start frame, last frame, rotation)
    for video_file in video_files:
        video_path = os.path.join(video_dir, video_file)
        print(f"Traitement de la vidéo : {video_path}")

        # Récupérer les actions de montage à effectuer pour la vidéo
        montage_actions = cv2_actions_to_operate(video_path, play_speed)
        starting_game_frame = montage_actions.get('Start_frame', 0)
        last_game_frame = montage_actions.get('Last_frame', None)
        rotation_state = montage_actions.get('Rotation_state', 0)


        # Stocker le frame de début du match et la rotation dans un df pandas temporaire pour les étapes suivantes du pipeline
        match_info_df.loc[len(match_info_df)] = {
            "video_path": video_path,
            "starting_game_frame": starting_game_frame,
            "last_game_frame": last_game_frame,
            "output_dir": output_dir,
            "rotation_state": rotation_state,
        }


In [15]:
match_info_df =pre_match_editing(
    video_dir=r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon',
    play_speed=2.0
)

match_info_df

Traitement de la vidéo : C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon\demi_finale_S2_250_Dijon_novembre_2025.mp4
Index du dernier frame : 42861
Début du match marqué au frame 132, soit 4.63 secondes
Traitement de la vidéo : C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon\finale_S2_250_Dijon_novembre_2025.mp4
Index du dernier frame : 39333
Début du match marqué au frame 107, soit 4.20 secondes


,video_path,starting_game_frame,last_game_frame,output_dir,rotation_state
0,C:\Users\habib\Desktop\Montages volley et beac...,132,42861,None,90
1,C:\Users\habib\Desktop\Montages volley et beac...,107,39333,None,270


In [7]:
from montage_auto_frames_gpu import video_rotation

video_rotation(
    video_path=r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon\demi_finale_S2_250_Dijon_novembre_2025.mp4',
    rotation_state=90,
)

Vidéo pivotée enregistrée : C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon\demi_finale_S2_250_Dijon_novembre_2025_rotated_90.mp4


In [20]:
import subprocess
video_path = r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs preprocess\matchs_bruts\barrage_leuven.mp4'
start_frame = 1000
last_frame = 2000
output_video = r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs preprocess\matchs_bruts\barrage_leuven_started.mp4'

In [ ]:
vf = f"select='between(n,{start_frame},{end_frame})',setpts=PTS-STARTPTS"

cmd = [
    "ffmpeg", "-y",
    "-hwaccel", "cuda",
    "-hwaccel_output_format", "cuda",
    "-i", video_path,
    "-vf", vf,
    "-an",
    "-c:v", "h264_nvenc",
    "-preset", "p4",
    "-rc", "constqp",
    "-qp", "18",
    output_video
]

print(f"[GPU] Extraction frames {start_frame} → {end_frame}")
print("      Command:", " ".join(cmd))

subprocess.run(cmd, check=True)

[GPU] Extraction frames 1000 → 2000
      Command: ffmpeg -y -hwaccel cuda -hwaccel_output_format cuda -i C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs preprocess\matchs_bruts\barrage_leuven.mp4 -vf select='between(n,1000,2000)',setpts=PTS-STARTPTS -an -c:v h264_nvenc -rc constqp -qp 18 C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs preprocess\matchs_bruts\barrage_leuven_started.mp4


CalledProcessError: Command '['ffmpeg', '-y', '-hwaccel', 'cuda', '-hwaccel_output_format', 'cuda', '-i', 'C:\\Users\\habib\\Desktop\\Montages volley et beach\\Jade&Math\\matchs preprocess\\matchs_bruts\\barrage_leuven.mp4', '-vf', "select='between(n,1000,2000)',setpts=PTS-STARTPTS", '-an', '-c:v', 'h264_nvenc', '-rc', 'constqp', '-qp', '18', 'C:\\Users\\habib\\Desktop\\Montages volley et beach\\Jade&Math\\matchs preprocess\\matchs_bruts\\barrage_leuven_started.mp4']' returned non-zero exit status 2880417800.